In [102]:
import pandas as pd
import numpy as np
from os import path
from CSVUtils import *
import ta
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.stats import zscore

In [56]:
def exponential_moving_avg(df: pd.DataFrame, price_col='Price', window_size=15, center=True):
    if center == True:
        ema_df = df[price_col].shift(int(window_size/2)).ewm(span=window_size).mean()
        return _remove_trailing_data(ema_df, window_size)
    else:
        ema_df = pd.Series.ewm(df[price_col], span=window_size).mean()
        return ema_df
    
def _remove_trailing_data(df: pd.DataFrame, window_size):
    return df.shift(-int(window_size/2)*2).shift(int(window_size/2))

In [61]:
DIR = "./input/yahoo"
nameList = ["RSX", "EWH", "IVV"]
trainStartDate = pd.to_datetime("2000-01-01")
trainEndDate = pd.to_datetime("2014-12-31")
testStartDate = pd.to_datetime("2015-01-01")
testEndDate = pd.to_datetime("2019-12-31")
df_list = []

for name in nameList:
#     df = csv2df(csv_path=DIR, csv_name=nameList[2]+".csv", source="yahoo")
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source="yahoo")
    df_list.append(df)

In [105]:
# df = df_list[2]
window_size = 15
for df in df_list:
    df['EMA'] = df['Price'].ewm(span=window_size).mean()
    df['MACD_diff'] = ta.trend.macd_diff(df['Price'])
    macd_direction = df['MACD_diff']/np.abs(df['MACD_diff'])
    df['MACD_change'] = (-1*macd_direction*macd_direction.shift(1)+1)/2
    
    delta_time = []
    for i in df['MACD_change']:
        if len(delta_time) == 0:
            result = 0
        elif i==0:
            result = delta_time[-1]+1
        else: #Nan or 1
            result = 0
        delta_time.append(result)
    df['delta_time'] = delta_time
    df = df.drop(['Open','High','Low','Price','Vol'], axis=1)
    df = df.dropna()

In [106]:
df

,Date,Change,EMA,MACD_diff,MACD_change,delta_time
34,2000-07-10,-0.001690,100.633190,0.002982,1.0,0
35,2000-07-11,0.003808,100.806802,0.055649,0.0,1
36,2000-07-12,0.005480,101.028754,0.116560,0.0,2
37,2000-07-13,0.003354,101.265859,0.165460,0.0,3
38,2000-07-14,0.009296,101.593215,0.242989,0.0,4
...,...,...,...,...,...,...
4967,2020-02-19,0.004789,335.249885,0.464364,0.0,7
4968,2020-02-20,-0.004090,335.654898,0.308837,0.0,8
4969,2020-02-21,-0.010281,335.574287,-0.056281,1.0,0
4970,2020-02-24,-0.033372,334.106249,-1.024897,0.0,1


In [79]:
(df['MACD_change']==0).astype(int)


0       0
1       0
2       0
3       0
4       0
       ..
4967    1
4968    1
4969    0
4970    1
4971    1
Name: MACD_change, Length: 4972, dtype: int32

In [75]:
macd_change = (-1*macd_direction*macd_direction.shift(1)+1)/2
macd_change

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4967    0.0
4968    0.0
4969    1.0
4970    0.0
4971    0.0
Name: MACD_diff, Length: 4972, dtype: float64

SyntaxError: invalid syntax (<ipython-input-89-1bef20162f5b>, line 1)